In [1]:
import os
import glob
import json, fire, re
from pathlib import Path
import io
import time

def is_export(cell):
    if cell['cell_type'] != 'code': 
        return False
    src = cell['source']
    if len(src) == 0 or len(src[0]) < 7: 
        return False
    return re.match(r'^\s*#\s*export\s*$', src[0], re.IGNORECASE) is not None
        
def notebook2script(f_name):
    "Finds cells starting with `#export` and puts them into a new module"
    f_name = Path(f_name)
    main_dic = json.load(open(f_name,'r',encoding="utf-8"))
    code_cells = [c for c in main_dic['cells'] if is_export(c)]
    module = '# ---------------------------------------------\n' +\
             '# | THIS FILE WAS AUTOGENERATED! DO NOT EDIT! |\n' +\
             '# ---------------------------------------------\n' +\
            f'# edit notebooks/{f_name.name} and run generate_all.py\n\n'
                
    if not (Path('scripts')).exists(): 
        ('scripts').mkdir()
    output_path = 'scripts/' + f'{f_name.stem[3:]}.py'
    
    for cell in code_cells: 
        module += ''.join(cell['source'][1:]) + '\n\n'
    module = re.sub(r' +$', '', module, flags=re.MULTILINE)
    
    with io.open(output_path, "w", encoding="utf-8") as f:
        f.write(module[:-2])
        
    print(f"{str(f_name):35} -> {str(output_path)}")

In [2]:
def clean():
    files = glob.glob('scripts/*')
    for f_name in files:
        if f_name.endswith('py'):
            os.remove(f_name)

def generate():
    for f_name in sorted(glob.glob('notebooks/*')):
        if f_name.endswith('ipynb'):
            notebook2script(f_name)

In [3]:
clean()
generate()

notebooks/00_utils.ipynb            -> scripts/utils.py
notebooks/01_parameter.ipynb        -> scripts/parameter.py
notebooks/02_data_block.ipynb       -> scripts/data_block.py
notebooks/03_data_bunch.ipynb       -> scripts/data_bunch.py
notebooks/04_augmentation.ipynb     -> scripts/augmentation.py
notebooks/05_initialization.ipynb   -> scripts/initialization.py
notebooks/06_loss.ipynb             -> scripts/loss.py
notebooks/07_model.ipynb            -> scripts/model.py
notebooks/08_linear.ipynb           -> scripts/linear.py
notebooks/09_convolution.ipynb      -> scripts/convolution.py
notebooks/10_pooling.ipynb          -> scripts/pooling.py
notebooks/11_batch_norm.ipynb       -> scripts/batch_norm.py
notebooks/12_other_layers.ipynb     -> scripts/other_layers.py
notebooks/13_sub_model.ipynb        -> scripts/sub_model.py
notebooks/14_optimizer.ipynb        -> scripts/optimizer.py
notebooks/15_training.ipynb         -> scripts/training.py
notebooks/16_callback.ipynb         -> scri